# 🧭 Guide interactif - variance de blocs, de dispersions et d'erreurs

## 🧩 SCÉNARIO 1 : Introduction à la variance de blocs

### 🎯 But pédagogique
Montrer comment la dépendance spatiale s'atténue avec la taille du support (**effet de support**).

---

### 🎓 Objectif

Montrer l’effet de la taille de support (bloc) sur la variance, comparée à :

- la **variance ponctuelle** ;
- la **variance de bloc théorique** (calculée à partir de l'intégrale de la fonction de covariance) ;
- la **variance de bloc expérimentale**, obtenue :
  - par **échantillonnage aléatoire** ;
  - ou par **moyenne sur sous-blocs**.

---

### 🔍 Concepts clés
- **Effet de support** : réduction de la variance lorsque la taille du bloc augmente.
- **Covariance spatiale** : mesure de la dépendance entre valeurs en fonction de la distance.
- **Variance de bloc** : mesure de la variabilité moyenne sur une surface ou un volume donné.

---



In [48]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
from numpy.fft import fft2, ifft2, fftshift
from scipy.stats import norm

def normal_score_transform(data):
    data = np.asarray(data).ravel()
    n = len(data)
    
    # Trier les données et garder les indices originaux
    sorted_idx = np.argsort(data)
    sorted_data = data[sorted_idx]
    
    # Quantiles empiriques (plotting positions)
    probs = (np.arange(1, n+1) - 0.5) / n  # méthode moyenne
    
    # Valeurs normales correspondantes
    norm_scores = norm.ppf(probs)
    
    # Tableau pour reconstruire dans l'ordre original
    result = np.empty_like(data, dtype=float)
    result[sorted_idx] = norm_scores
    
    return result
    
# --- Modèles de covariance ---
def spherical_covariance(h, range_, sill):
    cov = np.zeros_like(h)
    mask = h <= range_
    hr = h[mask] / range_
    cov[mask] = sill * (1 - 1.5 * hr + 0.5 * hr**3)
    return cov

def exponential_covariance(h, range_, sill):
    return sill * np.exp(-3 * h / range_)

def gaussian_covariance(h, range_, sill):
    return sill * np.exp(-3 * (h / range_)**2)
    
def get_covariance_model(model_name):
    if model_name == 'Sphérique':
        return spherical_covariance
    elif model_name == 'Exponentiel':
        return exponential_covariance
    elif model_name == 'Gaussien':
        return gaussian_covariance
    else:
        raise ValueError("Modèle inconnu")

def anisotropic_distance(X, Y, range_x, range_y, angle_deg=0):
    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    
    # Rotation + mise à l'échelle inverse des portées
    X_rot = (X * cos_a + Y * sin_a) / range_x
    Y_rot = (-X * sin_a + Y * cos_a) / range_y
    
    return np.sqrt(X_rot**2 + Y_rot**2)

# --- Génération FFT-MA anisotrope ---
def fftma_simulation(size, range_x, range_y, sill, nugget, angle_deg, model_name, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)

    h_aniso = anisotropic_distance(X, Y, range_x, range_y, angle_deg)
    cov_func = get_covariance_model(model_name)
    cov_model = cov_func(h_aniso, range_=1.0, sill=sill)
    cov_model += nugget * (X == 0) * (Y == 0)  # effet de pépite

    cov_fft = fft2(fftshift(cov_model))
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))

    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

# --- Agrégation par blocs carrés ---
def aggregate(field, block_size):
    if block_size == 0:
        return field
    s = field.shape[0]
    reduced_size = s // block_size
    # Découper le champ en blocs : (reduced_size, block_size, reduced_size, block_size)
    reshaped = field[:reduced_size*block_size, :reduced_size*block_size].reshape(
        reduced_size, block_size, reduced_size, block_size)
    # Moyenne sur les dimensions des blocs
    aggregated = reshaped.mean(axis=(1, 3))
    return aggregated

# --- Variance expérimentale des blocs ---
def variance_blocks(field, max_block_size):
    s = field.shape[0]
    variances = []
    block_sizes = list(range(0, max_block_size + 1, 2))
    for bsize in block_sizes:
        agg = aggregate(field, bsize)
        variances.append(np.var(agg))
    return block_sizes, np.array(variances)

# --- Variance théorique d'un bloc carré dans modèle sphérique ---
def theoretical_block_variance(range_x, range_y, sill, nugget, block_size, pixel_size=1, angle_deg=0, model_name = 'Sphérique'):
    if block_size == 0:
        return sill + nugget

    # Discrétisation du bloc [0, block_size] x [0, block_size]
    n = int(block_size / pixel_size)
    x = np.linspace(0, block_size, n, endpoint=False)
    X, Y = np.meshgrid(x, x)

    # Coordonnées en 2D pour tous les points
    points = np.stack([X.ravel(), Y.ravel()], axis=1)

    # Calcul de la matrice des distances anisotropes entre tous les points du bloc
    n_pts = len(points)
    dx = points[:, 0].reshape(-1, 1) - points[:, 0].reshape(1, -1)
    dy = points[:, 1].reshape(-1, 1) - points[:, 1].reshape(1, -1)
    h_aniso = anisotropic_distance(dx, dy, range_x, range_y, angle_deg)

    # Covariance entre tous les couples de points
    cov_func = get_covariance_model(model_name)
    cov = cov_func(h_aniso, range_=1.0, sill=sill)

    # Moyenne de la covariance (variance du bloc)
    return np.mean(cov) + nugget/(block_size*block_size)

# --- Paramètres fixes ---
size = 500           # taille du champ (500x500 pixels)
seed = 4263            # seed pour la simulation

# --- Fonction principale interactive ---
def interactive_variance(support, range_x, range_y, sill, nugget, angle_deg, model_name):
    if support > 50: support = 50
    
    field = fftma_simulation(
        size=size,
        range_x=range_x,
        range_y=range_y,
        sill=sill,
        nugget=nugget,
        angle_deg=angle_deg,
        model_name=model_name,
        seed=seed
    )
    field = normal_score_transform(field).reshape(field.shape)*np.sqrt(sill+nugget)
    
    plt.figure(figsize=(14,6))

    # Agrégation du champ
    agg = aggregate(field, support)

    # Calcul adaptatif de la légendre de couleur
    variance = sill + nugget
    std_dev = np.sqrt(variance)
    vmin = norm.ppf(0.05, loc=0, scale=std_dev)
    vmax = norm.ppf(0.95, loc=0, scale=std_dev)

    # Affichage du champ agrégé
    plt.subplot(1,2,1)
    plt.imshow(agg, cmap='viridis', origin='lower', vmin=vmin, vmax=vmax)
    plt.title(f'Champ agrégé, support = {support}x{support}')
    plt.colorbar(label='Valeurs')
    plt.axis('off')

    # Calcul variance expérimentale pour tous supports ≤ support max (50)
    max_support = 50
    bsizes, var_exp = variance_blocks(field, max_support)

    # Calcul variance théorique
    var_theo = [
        theoretical_block_variance(
            range_x=range_x, range_y=range_y, sill=sill,
            nugget = nugget, block_size=b, pixel_size=0.05*b,
            angle_deg=angle_deg, model_name = model_name
        )
        for b in bsizes
    ]

    # Affichage variance expérimentale vs théorique
    plt.subplot(1,2,2)
    plt.plot(bsizes, var_exp, 'o-', label='Variance expérimentale')
    plt.plot(bsizes, var_theo, 's--', label='Variance théorique')
    plt.axvline(support, color='red', linestyle=':', label=f'Support sélectionné = {support}')
    plt.xlabel('Taille du support (pixels)')
    plt.ylabel('Variance')
    plt.title('Variance des blocs vs taille de support')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.xlim(0, 50)
    plt.ylim(0, sill + nugget)
    plt.tight_layout()
    plt.show()

# --- Widgets ---
interact(
    interactive_variance,
    support=IntSlider(min=0, max=50, step=1, value=1, description='Support (pixels)'),
    range_x=FloatSlider(min=1, max=50, step=1, value=15, description='Portée X ($a_x)'),
    range_y=FloatSlider(min=1, max=50, step=1, value=15, description='Portée Y ($a_y)'),
    sill=FloatSlider(min=0.1, max=10, step=0.1, value=1.0, description='$c_1$'),
    nugget=FloatSlider(min=0, max=1, step=0.01, value=0, description='Effet de pépite ($c_0)'),
    angle_deg=FloatSlider(min=0, max=180, step=1, value=30, description='Angle ($θ$)'),
    model_name=Dropdown(options=['Sphérique', 'Exponentiel', 'Gaussien'], value='Sphérique', description='Modèle')
)

interactive(children=(IntSlider(value=1, description='Support (pixels)', max=50), FloatSlider(value=15.0, desc…

<function __main__.interactive_variance(support, range_x, range_y, sill, nugget, angle_deg, model_name)>

## 🧩 SCÉNARIO 2 : Discrétisation de la variance de bloc

### 🎯 But pédagogique  
Étudier l’influence de la discrétisation spatiale sur la précision du calcul de la variance de blocs à partir d’un modèle de covariance.

### ⚙️ Impact de la discrétisation

Dans la pratique numérique, cette intégrale double est approchée par une discrétisation spatiale du bloc en un nombre fini de points. La précision du calcul de la variance de bloc dépend donc de la résolution choisie :  
- Une faible résolution (peu de points) entraîne une approximation grossière et une estimation moins précise de la variance.  
- Une résolution élevée (beaucoup de points) améliore la précision mais augmente le coût de calcul.  

Ce scénario permet d’étudier quantitativement cet impact, en observant comment la variance estimée varie avec la densité de points de discrétisation.




In [49]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Covariances --- #
def spherical_covariance(h, range_, sill):
    cov = np.zeros_like(h)
    mask = h <= range_
    hr = h[mask] / range_
    cov[mask] = sill * (1 - 1.5 * hr + 0.5 * hr**3)
    return cov

def exponential_covariance(h, range_, sill):
    return sill * np.exp(-3 * h / range_)

def gaussian_covariance(h, range_, sill):
    return sill * np.exp(-3 * (h / range_)**2)

def get_covariance_model(name):
    return {
        'sphérique': spherical_covariance,
        'exponentiel': exponential_covariance,
        'gaussien': gaussian_covariance
    }[name]

# --- Distance anisotrope ---
def anisotropic_distance(dx, dy=0, dz=0, range_x=1, range_y=1, range_z=1, angle_deg=0):
    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)

    dx_rot = cos_a * dx + sin_a * dy
    dy_rot = -sin_a * dx + cos_a * dy
    dz_rot = dz  # Pas de rotation en Z

    return np.sqrt((dx_rot / range_x)**2 + (dy_rot / range_y)**2 + (dz_rot / range_z)**2)

# --- Calcul variance de bloc ---
def theoretical_block_variance_block(geometry, size, resolution, sill, range_x, range_y, range_z, angle_deg, model_name):
    if resolution < 2:
        return sill, None, None, None

    x = np.linspace(0, size, resolution)
    cov_func = get_covariance_model(model_name)

    if geometry == 'ligne':
        dx = x[:, None] - x[None, :]
        h = anisotropic_distance(dx, 0, 0, range_x, range_y, range_z, angle_deg)
        cov = cov_func(h, range_=1.0, sill=sill)
        return np.mean(cov), x, None, None

    elif geometry == 'surface':
        X, Y = np.meshgrid(x, x)
        dx = X.ravel()[:, None] - X.ravel()[None, :]
        dy = Y.ravel()[:, None] - Y.ravel()[None, :]
        h = anisotropic_distance(dx, dy, 0, range_x, range_y, range_z, angle_deg)
        cov = cov_func(h, range_=1.0, sill=sill)
        return np.mean(cov), X, Y, None

    elif geometry == 'cube':
        X, Y, Z = np.meshgrid(x, x, x)
        dx = X.ravel()[:, None] - X.ravel()[None, :]
        dy = Y.ravel()[:, None] - Y.ravel()[None, :]
        dz = Z.ravel()[:, None] - Z.ravel()[None, :]
        h = anisotropic_distance(dx, dy, dz, range_x, range_y, range_z, angle_deg)
        cov = cov_func(h, range_=1.0, sill=sill)
        return np.mean(cov), X, Y, Z

# --- Interface utilisateur ---
geometry_input = widgets.Dropdown(options=['ligne', 'surface', 'cube'], value='surface', description='Géométrie')
block_size_input = widgets.FloatText(value=10, description='Taille bloc')
resolution_input = widgets.IntText(value=5, description='Résolution')
max_resolution_input = widgets.IntText(value=50, description='Résolution max')
sill_input = widgets.FloatText(value=1.0, description='Sill')
range_x_input = widgets.FloatText(value=30, description='Portée X')
range_y_input = widgets.FloatText(value=30, description='Portée Y')
range_z_input = widgets.FloatText(value=30, description='Portée Z')
angle_input = widgets.IntText(value=0, description='Angle (°)')
model_input = widgets.Dropdown(options=['sphérique', 'exponentiel', 'gaussien'], value='sphérique', description='Modèle')

calc_button = widgets.Button(description="Calculer", button_style='success')
output = widgets.Output()

def on_calc_clicked(b):
    with output:
        clear_output()
        
        geometry = geometry_input.value
        block_size = block_size_input.value
        resolution = resolution_input.value
        max_resolution = max_resolution_input.value
        sill = sill_input.value
        range_x = range_x_input.value
        range_y = range_y_input.value
        range_z = range_z_input.value
        angle_deg = angle_input.value
        model_name = model_input.value
        
        # Calcul variance pour plusieurs résolutions
        res_list = np.arange(2, max_resolution + 1)
        variances = []
        for res in res_list:
            var, _, _, _ = theoretical_block_variance_block(
                geometry, block_size, res,
                sill, range_x, range_y, range_z,
                angle_deg, model_name)
            variances.append(var)
        variances = np.array(variances)
        
        # Variance et discrétisation pour la résolution choisie
        var_current, X, Y, Z = theoretical_block_variance_block(
            geometry, block_size, resolution,
            sill, range_x, range_y, range_z,
            angle_deg, model_name)
        
        print(f"Variance de bloc estimée à résolution {resolution} : {var_current:.6f}")
        
        # Affichage graphique
        fig = plt.figure(figsize=(14,6))
        
        # Graphe variance vs résolution
        ax1 = fig.add_subplot(1, 2, 1)
        ax1.plot(res_list, variances, 'b-', label='Variance vs résolution')
        ax1.plot(resolution, var_current, 'ro', label=f'Résolution choisie = {resolution}')
        ax1.set_xlabel('Résolution (nombre de points)')
        ax1.set_ylabel('Variance de bloc estimée')
        ax1.set_title('Variance de bloc selon la résolution')
        ax1.grid(True)
        ax1.legend()
        
        # Discrétisation (points du bloc)
        ax2 = fig.add_subplot(1, 2, 2, projection='3d' if geometry=='cube' else None)
        
        if geometry == 'ligne':
            ax2.plot(X, np.zeros_like(X), 'o')
            ax2.set_xlim(0, block_size)
            ax2.set_ylim(-1, 1)
            ax2.set_title("Discrétisation de la ligne")
            ax2.set_xlabel("X")
        elif geometry == 'surface':
            ax2.plot(X, Y, 'ko')
            ax2.set_aspect('equal')
            ax2.set_title("Discrétisation de la surface")
            ax2.set_xlabel("X")
            ax2.set_ylabel("Y")
        else:  # cube
            ax2.scatter(X, Y, Z, c='k')
            ax2.set_title("Discrétisation du cube")
            ax2.set_xlabel("X")
            ax2.set_ylabel("Y")
            ax2.set_zlabel("Z")
        
        plt.tight_layout()
        plt.show()

calc_button.on_click(on_calc_clicked)

ui = widgets.VBox([
    geometry_input, block_size_input, resolution_input, max_resolution_input,
    sill_input, range_x_input, range_y_input, range_z_input,
    angle_input, model_input,
    calc_button,
    output
])

display(ui)



## 🧩 SCÉNARIO 3 : Calculateur de variance de blocs

### 🎯 But pédagogique  
Outil informatique pour calculer la variance de bloc.

### 📐 Variance théorique d’un bloc carré

La variance d’un bloc de volume $V$ est donnée par :

$$
\sigma_V^2 = \frac{1}{V^2} \iint_V C(\| \mathbf{r} - \mathbf{r}' \|) \, d\mathbf{r} \, d\mathbf{r}'
$$

où :  
- $V$ est le volume du bloc,  
- $\mathbf{r}$ et $\mathbf{r}'$ sont des positions dans ce volume,  
- $C(h)$ est la fonction de covariance, avec $h = \| \mathbf{r} - \mathbf{r}'$ \| la distance entre les deux points.

### ⚙️ Description des paramètres

- **Dimension (dim)** : dimension spatiale du bloc (1 pour ligne, 2 pour surface, 3 pour cube).  
- **Plateau ($c_1$)** : variance structurée maximale du modèle de covariance.  
- **Effet de pépite ( $c_0$ )** : variance indépendante du modèle, représentant la variance microscopique ou bruit blanc.  
- **Portée en X, Y, Z ($a_x$, $a_y$, $a_z$)** : paramètres de portée (range) dans chaque direction définissant la portée spatiale de la corrélation.  
- **Longueur du bloc en X, Y, Z ($l_x$, $l_y$, $l_z$)** : dimensions physiques du bloc.  
- **Modèle de covariance** : choix du modèle théorique (sphérique, exponentiel, gaussien).

### 🚨 Limitations  
Le bloc est supposé orienté selon les directions d’anisotropie définies par les portées $a_x$, $a_y$, et $a_z$.  
*Note* : Si le bloc n’est pas orienté selon ces directions, il faut appliquer une rotation pour transformer le système en un cadre isotrope.





In [41]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import FloatText, Dropdown, Layout
# --- Covariances --- #
def spherical_covariance(h, range_, sill):
    cov = np.zeros_like(h)
    mask = h <= range_
    hr = h[mask] / range_
    cov[mask] = sill * (1 - 1.5 * hr + 0.5 * hr**3)
    return cov

def exponential_covariance(h, range_, sill):
    return sill * np.exp(-3 * h / range_)

def gaussian_covariance(h, range_, sill):
    return sill * np.exp(-3 * (h / range_)**2)

def get_covariance_model(name):
    return {
        "Sphérique": spherical_covariance,
        "Exponentiel": exponential_covariance,
        "Gaussien": gaussian_covariance
    }[name]

# --- Distance anisotrope --- #
def anisotropic_distance(dx, dy, dz, ax, ay, az):
    return np.sqrt((dx/ax)**2 + (dy/ay)**2 + (dz/az)**2)

# --- Calcul de variance de bloc --- #
def block_variance(dim, sill, nugget, ax, ay, az, lx, ly, lz, model_name, n_points=20):
    x = np.linspace(0, lx, n_points)
    cov_func = get_covariance_model(model_name)

    if dim == 1:
        X = x[:, None]
        dX = X - X.T
        h = anisotropic_distance(dX, 0, 0, ax, ay, az)

    elif dim == 2:
        y = np.linspace(0, ly, n_points)
        X, Y = np.meshgrid(x, y)
        coords = np.stack([X.ravel(), Y.ravel()], axis=1)
        dx = coords[:, 0][:, None] - coords[:, 0][None, :]
        dy = coords[:, 1][:, None] - coords[:, 1][None, :]
        h = anisotropic_distance(dx, dy, 0, ax, ay, az)

    elif dim == 3:
        y = np.linspace(0, ly, n_points)
        z = np.linspace(0, lz, n_points)
        X, Y, Z = np.meshgrid(x, y, z)
        coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
        dx = coords[:, 0][:, None] - coords[:, 0][None, :]
        dy = coords[:, 1][:, None] - coords[:, 1][None, :]
        dz = coords[:, 2][:, None] - coords[:, 2][None, :]
        h = anisotropic_distance(dx, dy, dz, ax, ay, az)

    else:
        raise ValueError("Dimension doit être 1, 2 ou 3")

    C = cov_func(h, range_=1.0, sill=sill)
    np.fill_diagonal(C, C.diagonal() + nugget)
    return np.mean(C)

# --- Widgets à saisie manuelle --- #
label_width = '120px'  # plus large que la valeur par défaut (~120px)

dim_input = Dropdown(options=[1, 2, 3], value=2, description='Dimension', style={'description_width': label_width})
sill_input = FloatText(value=1.0, description='$c_1$', style={'description_width': label_width})
nugget_input = FloatText(value=0.0, description='Effet de pépite ($c_0$)', style={'description_width': label_width})
ax_input = FloatText(value=30, description='Portée X ($a_x$)', style={'description_width': label_width})
ay_input = FloatText(value=30, description='Portée Y ($a_y$)', style={'description_width': label_width})
az_input = FloatText(value=30, description='Portée Z ($a_z$)', style={'description_width': label_width})
lx_input = FloatText(value=10, description='Longueur X ($l_x$)', style={'description_width': label_width})
ly_input = FloatText(value=10, description='Longueur Y ($l_y$)', style={'description_width': label_width})
lz_input = FloatText(value=10, description='Longueur Z ($l_z$)', style={'description_width': label_width})
model_input = Dropdown(
    options=["Sphérique", "Exponentiel", "Gaussien"], 
    value="Sphérique", 
    description='Modèle', 
    style={'description_width': label_width}
)

# --- Bouton de calcul --- #
calc_button = widgets.Button(description="Calculer la variance", button_style='success')
output_area = widgets.Output()

# --- Action quand on clique sur le bouton --- #
def on_calculate_clicked(b):
    with output_area:
        clear_output()
        var = block_variance(
            dim_input.value,
            sill_input.value,
            nugget_input.value,
            ax_input.value,
            ay_input.value,
            az_input.value,
            lx_input.value,
            ly_input.value,
            lz_input.value,
            model_input.value
        )
        print(f"▣ Variance de bloc : {var:.6f}")

calc_button.on_click(on_calculate_clicked)

# --- Affichage final --- #
ui = widgets.VBox([
    dim_input, sill_input, nugget_input,
    ax_input, ay_input, az_input,
    lx_input, ly_input, lz_input,
    model_input, calc_button, output_area
])
display(ui)